In [89]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from data_scraping.db_utils import get_connection

In [109]:
conn = get_connection()

# 1. Load data
market_df = pd.read_sql("SELECT * FROM price_history", conn)
cards_df = pd.read_sql("SELECT * FROM cards", conn)
# Merge price history with player metadata
full_df = market_df.merge(cards_df, on='card_id', how='left')


# Sort by player and time
full_df = market_df.sort_values(['card_id', 'date_time'])
# full_df = full_df[(full_df['league'] == 'Premier League') & (full_df['rating'] >= 84)]

print(full_df)

# Rolling averages for momentum
full_df['ma_short'] = full_df.groupby('card_id')['pc_value'].transform(lambda x: x.rolling(3).mean())
full_df['ma_long'] = full_df.groupby('card_id')['pc_value'].transform(lambda x: x.rolling(10).mean())
full_df['momentum'] = full_df['ma_short'] - full_df['ma_long']

# Rolling std for volatility
full_df['volatility'] = full_df.groupby('card_id')['pc_value'].transform(lambda x: x.rolling(10).std())


C:\Users\shane\AppData\Local\Temp\ipykernel_21648\2461998958.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  market_df = pd.read_sql("SELECT * FROM price_history", conn)
C:\Users\shane\AppData\Local\Temp\ipykernel_21648\2461998958.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cards_df = pd.read_sql("SELECT * FROM cards", conn)


        card_id  pc_value  console_value           date_time
189032       37       0.0            NaN 2024-09-20 09:30:00
189392       37       NaN            0.0 2024-09-20 09:30:00
189033       37       0.0            NaN 2024-09-21 09:30:00
189393       37       NaN            0.0 2024-09-21 09:30:00
189034       37       0.0            NaN 2024-09-22 09:30:00
...         ...       ...            ...                 ...
596979    59957       NaN            0.0 2025-09-14 09:30:00
596849    59957       0.0            NaN 2025-09-15 09:30:00
596980    59957       NaN            0.0 2025-09-15 09:30:00
596850    59957       0.0            NaN 2025-09-16 09:30:00
596981    59957       NaN            0.0 2025-09-16 09:30:00

[643062 rows x 4 columns]


In [91]:
# Next-period price
full_df['price_next'] = full_df.groupby('card_id')['pc_value'].shift(-1)

# Target: price goes up
full_df['target'] = (full_df['price_next'] > full_df['pc_value']).astype(int)

# Drop NaNs caused by rolling/shift
full_df = full_df.dropna(subset=['momentum', 'volatility', 'target'])

In [92]:
start_date = full_df['date_time'].min()
train_cutoff = start_date + pd.Timedelta(weeks=3)
test_cutoff  = train_cutoff + pd.Timedelta(weeks=3)

# Train = first 4 weeks
train_df = full_df[full_df['date_time'] <= train_cutoff]

# Test = next 4 weeks only
test_df = full_df[(full_df['date_time'] > train_cutoff) & (full_df['date_time'] <= test_cutoff)]

print("Train:", train_df['date_time'].min(), "→", train_df['date_time'].max())
print("Test :", test_df['date_time'].min(), "→", test_df['date_time'].max())

Train: 2024-09-29 09:30:00 → 2024-10-19 10:30:00
Test : 2024-10-20 10:30:00 → 2024-11-09 10:30:00


In [93]:
feature_cols = ['momentum', 'volatility']

X_train, y_train = train_df[feature_cols], train_df['target']
X_test, y_test   = test_df[feature_cols], test_df['target']

In [94]:
model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

Accuracy: 0.42857142857142855
ROC-AUC: 0.5913461538461539


In [95]:
# Add predictions back to the test dataframe
test_results = test_df.copy()
test_results['pred_prob'] = y_prob
test_results['signal'] = 'HOLD'
test_results.loc[test_results['pred_prob'] > 0.6, 'signal'] = 'BUY'
test_results.loc[test_results['pred_prob'] < 0.4, 'signal'] = 'SELL'

# Now you can print name + predictions
print(test_results[['pred_prob', 'signal']].head(20))

        pred_prob signal
192662   0.197422   SELL
192663   0.197422   SELL
192664   0.197422   SELL
192665   0.197422   SELL
192666   0.197422   SELL
192667   0.197422   SELL
192668   0.197422   SELL
192669   0.762669    BUY
192670   0.131207   SELL
192671   0.259011   SELL
192672   0.259011   SELL
192673   0.259011   SELL
192674   0.259011   SELL
192675   0.259011   SELL
192676   0.259011   SELL
192677   0.259011   SELL
192678   0.108060   SELL
192679   0.517849   HOLD
192680   0.517849   HOLD
192681   0.197422   SELL
